In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import pip as p
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score, mean_squared_error
df = pd.read_csv("diamonds.csv")

# Eliminering av nullvärden
df.dropna(how="any", inplace=True) # Tar bort nullvärden

# Eliminering av negativa eller nollor i dimnesions värden
df = df[(df['x'] > 0) & (df['y'] > 0) & (df['z'] > 0)] # Ta bort nollor eller negativa värden

# Eliminering av felaktig depth-värden
df["depth_calc"] = 2 * (df["z"] / (df["x"] + df["y"])) * 100 # Beräkna depth enligt formel
df["depth_diff"] = np.abs(df["depth"] - df["depth_calc"]) # Skillnad mellan depth i dataframen mot beräknad depth
felaktiga_depthvärden = df[df["depth_diff"] > 1]# Visa de som skiljer sig med mer än 1 procent.
df = df.drop(felaktiga_depthvärden.index, errors='ignore') # Ta bort felaktiga_depthvärden från dataframe.

## KOPIERAT FRÅN GAMLA KUNSKAPSKONTROLLEN

df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z,depth_calc,depth_diff
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,61.286255,0.213745
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,59.767141,0.032859
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,56.896552,0.003448
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,62.396204,0.003796
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,63.291139,0.008861


In [3]:
X = df.drop(columns=["price"])
y = df["price"]


X = pd.get_dummies(X, drop_first=True)

print(X)
print(y)

       Unnamed: 0  carat  depth  table     x     y     z  depth_calc  \
0               1   0.23   61.5   55.0  3.95  3.98  2.43   61.286255   
1               2   0.21   59.8   61.0  3.89  3.84  2.31   59.767141   
2               3   0.23   56.9   65.0  4.05  4.07  2.31   56.896552   
3               4   0.29   62.4   58.0  4.20  4.23  2.63   62.396204   
4               5   0.31   63.3   58.0  4.34  4.35  2.75   63.291139   
...           ...    ...    ...    ...   ...   ...   ...         ...   
53935       53936   0.72   60.8   57.0  5.75  5.76  3.50   60.816681   
53936       53937   0.72   63.1   55.0  5.69  5.75  3.61   63.111888   
53937       53938   0.70   62.8   60.0  5.66  5.68  3.56   62.786596   
53938       53939   0.86   61.0   58.0  6.15  6.12  3.74   60.961695   
53939       53940   0.75   62.2   55.0  5.83  5.87  3.64   62.222222   

       depth_diff  cut_Good  ...  color_H  color_I  color_J  clarity_IF  \
0        0.213745     False  ...    False    False    False 

In [12]:
X_train, X_tet, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

lin_reg = LinearRegression(n_jobs=-1)
dec_reg = DecisionTreeRegressor()
ran_reg = RandomForestRegressor(n_jobs=-1)

lin_reg.fit(X_train, y_train)
dec_reg.fit(X_train, y_train)
ran_reg.fit(X_train, y_train)

y_pred_lin = lin_reg.predict(X_val)
y_pred_dec = dec_reg.predict(X_val)
y_pred_ran = ran_reg.predict(X_val)

print("LinearRegression MSE: ", mean_squared_error(y_val, y_pred_lin))
print("DecisionTree MSE: ", mean_squared_error(y_val, y_pred_dec))
print("RandomForrest MSE: ", mean_squared_error(y_val, y_pred_ran))

LinearRegression MSE:  1253333.4914634048
DecisionTree MSE:  712.477789815818
RandomForrest MSE:  580.4900602693084
